# Snowflake

> Extract Data from text using LLM

In [ ]:
#| default_exp db

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import List, Dict, Any
import snowflake.connector
import pandas as pd
import json
from llm_data_extractor.models import DBConfig

In [ ]:
#| export

def get_connection(config: DBConfig) -> snowflake.connector.SnowflakeConnection:
    """Establishes a connection to Snowflake."""
    conn_params = {
        "account": config.account,
        "user": config.user,
        "password": config.password,
        "warehouse": config.warehouse,
        "database": config.database,
        "schema": config.schema_,
    }
    if config.role:
        conn_params["role"] = config.role
        
    return snowflake.connector.connect(**conn_params)

In [ ]:
#| export

def fetch_data(query: str, conn: snowflake.connector.SnowflakeConnection) -> pd.DataFrame:
    """Fetches data from Snowflake using a given query."""
    cursor = conn.cursor()
    try:
        cursor.execute(query)
        df = cursor.fetch_pandas_all()
        return df
    finally:
        cursor.close()

In [ ]:
#| export

def insert_results(results: List[Dict[str, Any]], conn: snowflake.connector.SnowflakeConnection, table_name: str):
    """
    Inserts or merges extraction results into a Snowflake table.

    Args:
        results: List of dictionaries, where each dictionary represents a row to insert.
        conn: Snowflake connection object.
        table_name: The name of the table to insert results into.
    """
    if not results:
        return

    df = pd.DataFrame(results)
    
    # Use a temporary table for merging
    temp_table_name = f"temp_results_{pd.Timestamp.now().strftime('%Y%m%d%H%M%S')}"
    
    from snowflake.connector.pandas_tools import write_pandas
    
    # 1. Write to temporary table
    # Uppercase column names for Snowflake to avoid issues with case sensitivity
    df.columns = [c.upper() for c in df.columns]
    write_pandas(conn, df, temp_table_name.upper(), auto_create_table=True, overwrite=True)
    
    cursor = conn.cursor()
    try:
        # 2. Merge from temp table to target table
        merge_sql = f"""
        MERGE INTO {table_name} AS target
        USING {temp_table_name.upper()} AS source
        ON target.TABLE_NAME = source.TABLE_NAME
        AND target.FIELD_NAME = source.FIELD_NAME
        AND target.FIELD_ID = source.FIELD_ID
        AND target.QUESTION_ID = source.QUESTION_ID
        WHEN MATCHED THEN
            UPDATE SET target.results = PARSE_JSON(source.RESULTS)
        WHEN NOT MATCHED THEN
            INSERT (TABLE_NAME, FIELD_NAME, FIELD_ID, QUESTION_ID, results)
            VALUES (source.TABLE_NAME, source.FIELD_NAME, source.FIELD_ID, source.QUESTION_ID, PARSE_JSON(source.RESULTS))
        """
        cursor.execute(merge_sql)
    finally:
        # 3. Drop temporary table
        cursor.execute(f"DROP TABLE IF EXISTS {temp_table_name.upper()}")
        cursor.close()


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()